- Branched Topology
- 10MW 40개, 해상 변전소 후보는 9개 

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt

# 데이터 불러오기
- 10MW 터빈 총 40개 정의
- 보안 상의 이유로 데이터 및 시각화 결과는 삭제

In [ ]:
turbine_pos_10M = np.array() # 각 터빈들의 좌표 정보가 있는 2차원 행렬 정의의
turbine_pos_10M = np.delete(turbine_pos_10M, 0, axis=1)
turbine_pos_10M[:, 0:] /= 1000

In [ ]:
# 터빈 위치 시각화화
plt.scatter(turbine_pos_10M[:, 0], turbine_pos_10M[:, 1], color='C1', label='10MW')

for s in range(len(turbine_pos_10M)):
    plt.text(turbine_pos_10M[s, 0], turbine_pos_10M[s, 1], str(s+1))

plt.legend()
plt.title('Turbines')
plt.show()

In [ ]:
# 해상변전소 좌표 설정 (임의)
substations = []
sets = [(27, 28), (35, 36), (34, 35), (33, 34)] # 중점, 영역 안쪽 4개 substation
for (a, b) in sets:
    substations.append((turbine_pos_10M[a-1, :] + turbine_pos_10M[b-1, :])/2)

substations = np.array(substations)
new_x = (turbine_pos_10M[35, 0]*3 - turbine_pos_10M[34, 0])/2
new_y = (turbine_pos_10M[35, 1]*3 - turbine_pos_10M[34, 1])/2
substations = np.insert(substations, 1, np.array([new_x, new_y]), axis=0)
temp = substations[1:, :].copy()
temp[:, 0] += 2
temp[:, 1] -= 0.5
substations_10M = np.vstack((substations, temp))


# 육지변전소 좌표 설정 (임의)
onshore_sub = np.array([substations_10M[5, 0]+16, turbine_pos_10M[38, 1]])

# substations 시각화
plt.figure(figsize=(15, 5))
plt.scatter(turbine_pos_10M[:, 0], turbine_pos_10M[:, 1], color='C0', label='10MW')
plt.scatter(substations_10M[:, 0], substations_10M[:, 1],  color='C1', label='10MW_OFFSUB')
plt.scatter(onshore_sub[0], onshore_sub[1], color='C2', label='10MW_ONSUB')

for s in range(len(turbine_pos_10M)):
    plt.text(turbine_pos_10M[s, 0], turbine_pos_10M[s, 1], str(s+1))

for itm, s in enumerate(substations_10M):
    plt.text(s[0], s[1], f'SUB_{itm+1}')
plt.text(onshore_sub[0], onshore_sub[1], 'ON_SUB')
plt.legend( bbox_to_anchor=(1.3, 0.5))
plt.title('Turbines with 9 substations')
plt.show()

In [ ]:
nodes_10M = np.vstack((substations_10M, turbine_pos_10M)) # 상위 9개는 substation
num_nodes_10M = nodes_10M.shape[0]
num_sub_10M = substations_10M.shape[0]
power_production = {i:1 for i in range(9, num_nodes_10M)}
print(f'Number of 10MW turbines and substation : {num_nodes_10M}\nNumber of substations : {num_sub_10M}')

substations_index = list(range(9))
turbines_index = list(range(9,49))

Number of 10MW turbines and substation : 49
Number of substations : 9


# 모델 정의

\begin{align}
&\min \sum_{(i, j) \in A} \sum_{t \in T} c_{i,\, j}^{t} \, x_{i,\, j}^t + \sum_{k \in V_{0}} a_{k} u_{k} \quad (a_k = 180 \times dist(k,\ onshore\ sub),\ k \in V_0 )\\
&\sum_{t \in T} x_{i,\, j}^{t} = y_{i,\, j}, \quad (i, j) \in A \\ 
&\sum_{i \in V : i \ne h} \left( f_{h,\, i} - f_{i,\, h} \right) = P_{h}, \quad h \in V_{T} \\
&\sum_{t \in T} k_t \ x_{i,\, j}^{t} \ge f_{i,\, j}, \quad (i, j) \in A \\
&\sum_{j \in V : j \ne h} y_{h,\, j} = 1, \quad h \in V_{T} \\
&\sum_{j \in V : j \ne h} y_{h,\, j} = 0, \quad h \in V_{0} \\
&\sum_{k \in V_{0}} u_{k} = 1 \\
&y_{i, j} \le u_j, \quad i \in V_{T} , \; j \in V_{0} \\
\notag \\[10pt] 
&x_{i, j}^t \in \{0, 1 \}, \quad (i, j) \in A, \; t \in T \\
&y_{i,\, j} \in \{0, 1 \}, \quad (i, j) \in A \\
&f_{i,\, j} \ge 0, \quad (i, j) \in A \\
&u_{k} \in \{0, 1 \}, \quad k \in V_{0}
\end{align}

In [ ]:
model_10M = gp.Model('CableRoutingOptimization10M')

model_10M.setParam('TimeLimit', 3600) # Time limit
model_10M.setParam('DisplayInterval', 20)

cable_types = [1, 2, 3, 4, 5] # 5가지의 케이블 타입입
cable_capacity_8M = {1: 3.68, 2: 4.19, 3: 6.34, 4: 8.06, 5: 9}  #각 케이블 타입 당 Capacity
cable_cost = {1: 64, 2: 68, 3: 75, 4: 90, 5: 100} # 케이블 단위 길이당 COst, 다운 스케일링(1/1000)

off_on_cost = np.array([np.linalg.norm(substations_10M[i] - onshore_sub)* 180 for i in range(9)]) # 해상-육지 변전소 연결 비용

x = model_10M.addVars(num_nodes_10M, num_nodes_10M, cable_types, vtype=GRB.BINARY, name='x')
y = model_10M.addVars(num_nodes_10M, num_nodes_10M, vtype=GRB.BINARY, name='y')
f = model_10M.addVars(num_nodes_10M, num_nodes_10M, vtype=GRB.CONTINUOUS, lb=0, name='f')
u = model_10M.addVars(num_sub_10M, vtype=GRB.BINARY, name='u')

# 목적함수
# 식(1)
model_10M.setObjective(gp.quicksum(cable_cost[t] * np.linalg.norm(nodes_10M[i]-nodes_10M[j], ord=2) * x[i, j, t] 
                                  for i in range(num_nodes_10M) 
                                  for j in range(num_nodes_10M) 
                                  for t in cable_types if i!=j)\
                      + gp.quicksum(off_on_cost[k] * u[k] for k in range(num_sub_10M)), GRB.MINIMIZE)

# 제약조건
# 식(2)
for i in range(num_nodes_10M):
    for j in range(num_nodes_10M):
        if i!=j:
            model_10M.addConstr(gp.quicksum(x[i, j, t] for t in cable_types) == y[i, j], 
                               name=f'cable_type_selection_{i}_{j}')

# 식(3)
for h in range(9, num_nodes_10M):
    model_10M.addConstr(gp.quicksum(f[h, i] - f[i, h] for i in range(num_nodes_10M) if i!=h) == power_production[h], 
                       name=f'flow_conservation_{h}')

# 식(4)
for i in range(num_nodes_10M):
     for j in range(num_nodes_10M):
          if i!=j:
               model_10M.addConstr(gp.quicksum(cable_capacity_8M[t] * x[i, j, t] for t in cable_types) >= f[i, j], 
                                  name=f'capacity_constraint_{i}_{j}')

# 식(5)
for h in range(9, num_nodes_10M):
    model_10M.addConstr(gp.quicksum(y[h, j] for j in range(num_nodes_10M) if j!=h) == 1, name=f'one_outgoing_cable_{h}')

# 식(6)
for h in range(num_sub_10M):
    model_10M.addConstr(gp.quicksum(y[h, j] for j in range(num_nodes_10M) if j!=h) == 0, name='no_outgoing_cable_substation')

# 해상변전소 제약조건
model_10M.addConstr(gp.quicksum(u[i] for i in range(num_sub_10M)) == 1, name='one_offshore_substation')

# 해상변전소 제약조건 (2)
for i in range(num_nodes_10M):
    for j in range(num_sub_10M):
        if i!=j:
            model_10M.addConstr(y[i, j] <= u[j], 'one_incomming_substation')

# inside_sub = [0, 2, 3, 4]
# for i in inside_sub:
#     model_10M.addConstr(u[i] == 0)

model_10M.optimize()

Set parameter TimeLimit to value 3600
Set parameter DisplayInterval to value 20
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 5226 rows, 16816 columns and 35289 nonzeros
Model fingerprint: 0x79ce02e3
Variable types: 2401 continuous, 14415 integer (14415 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1585 rows and 4007 columns
Presolve time: 0.11s
Presolved: 3641 rows, 12809 columns, 27889 nonzeros
Variable types: 1920 continuous, 10889 integer (10889 binary)
Found heuristic solution: objective 15408.820413

Root relaxation: objective 6.620403e+03, 5457 iterations, 0.09 seconds (0.20 work units)

    Nodes    |    Current Node    |     Objec

In [ ]:
print(f'Optimal solution found with total cost : {model_10M.objVal}')
print(f'Number of decision variables : {model_10M.NumVars}')
print(f'Number of constraints : {model_10M.NumConstrs}')

connections = []
# 실제로 연결된 간선만 추출출
for i in range(num_nodes_10M):
    for j in range(num_nodes_10M):
        if i != j and y[i, j].x > 0.5:
            for t in cable_types:
                if x[i, j, t].x > 0.5:
                    connections.append((i, j, t))

Optimal solution found with total cost : 7405.226685791344
Number of decision variables : 16816
Number of constraints : 5226


# 최적화 결과 시각화

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(turbine_pos_10M[:, 0], turbine_pos_10M[:, 1], color='C0', label='10MW')
plt.scatter(substations_10M[:, 0], substations_10M[:, 1],  color='C1', label='10MW_OFFSUB')
plt.scatter(onshore_sub[0], onshore_sub[1], color='C2', label='10MW_ONSUB')

for s in range(len(turbine_pos_10M)):
    plt.text(turbine_pos_10M[s, 0], turbine_pos_10M[s, 1], str(s+1))

for itm, s in enumerate(substations_10M):
    plt.text(s[0], s[1], f'SUB_{itm+1}')
plt.text(onshore_sub[0], onshore_sub[1], 'ON_SUB')
cable_colors = {
    1: 'blue',
    2: 'green',
    3: 'orange',
    4: 'purple',
    5: 'red'
}

for (i, j, t) in connections:
    plt.plot([nodes_10M[i, 0], nodes_10M[j, 0]], [nodes_10M[i, 1], nodes_10M[j, 1]], color=cable_colors[t], label=f'cable_type_{t}')

plt.plot([nodes_10M[3, 0], onshore_sub[0]], [nodes_10M[3, 1], onshore_sub[1]], color = 'cyan', label='cable_type_6')

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1, 1))

plt.title('Optimized connections')
plt.show()    

In [9]:
connections

[(9, 13, 1),
 (10, 14, 1),
 (11, 15, 1),
 (12, 16, 1),
 (13, 17, 1),
 (14, 18, 1),
 (15, 19, 1),
 (16, 20, 1),
 (17, 21, 1),
 (18, 22, 1),
 (19, 23, 1),
 (20, 24, 1),
 (21, 25, 2),
 (22, 26, 2),
 (23, 27, 2),
 (24, 28, 2),
 (25, 29, 3),
 (26, 30, 3),
 (27, 31, 3),
 (28, 32, 3),
 (29, 33, 3),
 (30, 34, 3),
 (31, 35, 3),
 (32, 36, 3),
 (33, 37, 4),
 (34, 38, 4),
 (35, 39, 4),
 (36, 40, 4),
 (37, 42, 4),
 (38, 3, 4),
 (39, 3, 4),
 (40, 43, 4),
 (41, 45, 1),
 (42, 3, 5),
 (43, 3, 5),
 (44, 47, 1),
 (45, 46, 1),
 (46, 3, 1),
 (47, 3, 1),
 (48, 44, 1)]